In [358]:
import pandas as pd

In [359]:
import numpy as np
np.random.randint(1,361)

48

In [360]:
zodiac_dict = {
    1: 'Aries', 2: 'Taurus', 3: 'Gemini',4: 'Cancer',5: 'Leo',6: 'Virgo',7: 'Libra',8: 'Scorpio',9: 'Sagittarius',10: 'Capricorn',11: 'Aquarius',12: 'Pisces'
}

planet_pos = {
    'Asc':0,'Sun':1,'Moon':2,'Mercury':3,'Venus':4,'Mars':5,'Jupiter':6,'Saturn':7,'Rahu':8
}


base_val = 100

In [361]:
def get_avastha_val(p_deg):
    p_deg = p_deg % 30
    avastha_ranges = {
        (0, 6): "Bala (Infant)",
        (6, 12): "Kumara (Youth)",
        (12, 18): "Yuva (Adult)",
        (18, 24): "Vriddha (Old)",
        (24, 30): "Mrita (Dead)"
    }
    # Strength scores going from 80 to 100 and back to 80
    strength_scores = {
        "Bala (Infant)": lambda d: 80 + (d / 6) * 80,  # Increases from 80 to 100
        "Kumara (Youth)": lambda d: 100,  # Constant 100
        "Yuva (Adult)": lambda d: 100,  # Constant 100
        "Vriddha (Old)": lambda d: 100 - ((d - 18) / 6) * 80,  # Decreases from 100 to 80
        "Mrita (Dead)": lambda d: 80  # Constant 80
    }
    for range_, avastha_name in avastha_ranges.items():
        if range_[0] <= p_deg < range_[1]:
            strength = strength_scores[avastha_name](p_deg)
            return avastha_name, strength/100
        
get_avastha_val(359)

('Mrita (Dead)', 0.8)

In [362]:
planet_strength_variance = {
    'Jupiter': {'enemy_sign': [30, 60, 150, 180],
             'exalted': 95,
             'friendly_sign': [120, 90, 0, 210],
             'moola_trikona': [240, 250],
             'neutral_sign': [270, 300],
             'own_sign': [240, 330]},
 'Mars': {'enemy_sign': [60, 150],
          'exalted': 298,
          'friendly_sign': [120, 90, 240, 330],
          'moola_trikona': [0, 12],
          'neutral_sign': [30, 180, 270, 300],
          'own_sign': [0, 210]},
 'Mercury': {'enemy_sign': [0, 210, 240, 330],
             'exalted': 165,
             'friendly_sign': [120, 30, 180],
             'moola_trikona': [165, 170],
             'neutral_sign': [90],
             'own_sign': [60, 150]},
 'Moon': {'enemy_sign': [],
          'exalted': 33,
          'friendly_sign': [60, 120, 150],
          'moola_trikona': [33, 50],
          'neutral_sign': [0, 210, 240, 330, 180, 270, 300],
          'own_sign': [90]},
 'Saturn': {'enemy_sign': [0, 90, 120, 210],
            'exalted': 200,
            'friendly_sign': [30, 60, 150, 180],
            'moola_trikona': [300, 320],
            'neutral_sign': [240, 330],
            'own_sign': [270, 300]},
 'Sun': {'enemy_sign': [30, 180, 270, 300],
         'exalted': 10,
         'friendly_sign': [0, 90, 210, 240, 330],
         'moola_trikona': [120, 140],
         'neutral_sign': [60, 150],
         'own_sign': [120]},
 'Venus': {'enemy_sign': [120, 90],
           'exalted': 357,
           'friendly_sign': [240, 330, 0, 210],
           'moola_trikona': [180, 195],
           'neutral_sign': [0, 120, 240],
           'own_sign': [30, 180]}
           }
           

strength_multipliers = {
        'exalted': 1,  # Example multiplier for exalted planets
        'own_sign': 0.75,  # Example multiplier for planets in their own sign (Mooltrikona)
        'friendly_sign': 0.5,  # Example multiplier for planets in a friendly sign
        'neutral_sign': 0.25,  # Base strength for planets in a neutral sign
        'enemy_sign': 0.1,  # Reduced strength for planets in an enemy sign
        'none':0.2
    }

def is_within_range(degree, start, range_size=30):
    """Check if the degree is within the specified range."""
    end = (start + range_size) % 360
    if start < end:
        return start <= degree < end
    else:  # range wraps around the 360-degree point
        return start <= degree or degree < end

def calculate_planetary_strength(planet, degrees):
    """
    Calculate the strength of a planet given its name and degrees.
    
    Args:
    planet (str): The name of the planet.
    degrees (int): The degrees of the planet from 0 to 360.
    
    Returns:
    str: The category and strength value of the planet.
    """
    if planet not in planet_strength_variance:
        return "Invalid planet name"
    
    planet_data = planet_strength_variance[planet]
    
    if degrees == planet_data['exalted']:
        category = 'exalted'
    elif any(is_within_range(degrees, start) for start in planet_data['own_sign']):
        category = 'own_sign'
    elif any(is_within_range(degrees, start) for start in planet_data['friendly_sign']):
        category = 'friendly_sign'
    elif any(is_within_range(degrees, start) for start in planet_data['neutral_sign']):
        category = 'neutral_sign'
    elif any(is_within_range(degrees, start) for start in planet_data['enemy_sign']):
        category = 'enemy_sign'
    else:
        category = 'none'
    
    strength = strength_multipliers[category]
    return category, strength

calculate_planetary_strength('Moon', 359)

('neutral_sign', 0.25)

In [363]:
def calculate_rashi(longitude):
    """
    Calculates the Rashi (sign) based on the longitude.
    
    Args:
    - longitude (float): Longitude of the planet (0 to 360 degrees).
    
    Returns:
    - str: Name of the Rashi.
    """
    signs = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]
    index = int(longitude / 30)  # Each sign is 30 degrees wide
    return signs[index % 12]  # % 12 to handle if longitude > 360

def calculate_navamsa(longitude):
    """
    Calculates the Navamsa based on the longitude.
    
    Args:
    - longitude (float): Longitude of the planet (0 to 360 degrees).
    
    Returns:
    - str: Name of the Navamsa.
    """
    signs = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]
    navamsa_index = int((longitude/ 3.33333 % 12) )  # Each Navamsa is 3.33333 degrees wide
    return signs[navamsa_index]

def is_vargottama(p_deg):
    return calculate_rashi(longitude) == calculate_navamsa(longitude)
# Example usage:
longitude = 29  # Example: Longitude of the planet
rashi = calculate_rashi(longitude)
navamsa = calculate_navamsa(longitude)
print(f"Rashi: {rashi}")
print(f"Navamsa: {navamsa}")

print(is_vargottama(359))

Rashi: Aries
Navamsa: Sagittarius
False


In [364]:
def get_peak_value(p_name, p_deg):
    if(p_name in ['Asc', 'Rahu', 'Ketu']):
        return '', 1
    avastha = get_avastha_val(p_deg)
    vargottama_val = 2 if is_vargottama(p_deg)==True else 1
    val = calculate_planetary_strength(p_name, p_deg)
    # print(val, vargottama_val, avastha)

    return_val = f'Sign: {val[0]} {', vargottam' if vargottama_val==2 else ''}, Avastha: {avastha[0]}', 2 - val[1] * vargottama_val * avastha[1]
    # print (return_val)
    return  return_val

print(get_peak_value("Sun", 320))
    

# def calculate_chubby_v2(p1_deg, p2_deg, p1_name, p2_name, peak_val):
#     if(peak_val > 5):
#         asc_val = peak_val  - get_peak_value(p1_deg)/base_val
#         moon_val = peak_val  - get_peak_value(p2_deg)/base_val
#     else:
#         asc_val = peak_val  + get_peak_value(p1_deg)/base_val
#         moon_val = peak_val  + get_peak_value(p2_deg)/base_val
#     return (asc_val * moon_val) ** 0.5 

('Sign: enemy_sign , Avastha: Vriddha (Old)', 1.9266666666666667)


In [365]:
def calc_combination_strength(p1_name, p2_name, p1_deg, p2_deg, peak_val):
    p1_offset_val = get_peak_value(p1_name, p1_deg)
    p2_offset_val = get_peak_value(p2_name, p2_deg)
    if(peak_val > 5):
        p1_val = peak_val  - p1_offset_val[1]/10
        p2_val = peak_val  - p2_offset_val[1]/10
    else:
        p1_val = peak_val  + p1_offset_val[1]/10
        p2_val = peak_val + p2_offset_val[1]/10
    return f'{p1_name} - {p1_offset_val[0]}, {p2_name} - {p2_offset_val[0]}', (p1_val * p2_val) ** 0.5 


<function __main__.calculate_chubby_v2(p1_name, p2_name, p1_deg, p2_deg, peak_val)>

In [366]:
data_v2 = []

def combination2(planet1_name, planet2_name, first, second, start_val, skip_list, info):
    second_zn = (second - 1) % 12 + 1 
    if second_zn in skip_list:
        print(f'Skipping as {planet2_name} in {zodiac_dict[second_zn]}')
        return
    first_start = (first - 1)*30 
    first_end = (first)*30 + 1
    second_start = ((second -1)*30)%360
    second_end = ((second)*30)%360 + 1
    
    text = f'{info}. {planet2_name} in {zodiac_dict[second_zn]} and {planet1_name} in {zodiac_dict[first]}'
    print(first_start,first_end, second_start, second_end, start_val, text)
    fill_data(planet1_name, planet2_name,start_val, text, first_start, first_end, second_start, second_end)
   
def fill_data(planet1_name:str, planet2_name, start_val, text, first_start, first_end, second_start, second_end):
    p1 = planet_pos[planet1_name]
    p2 = planet_pos[planet2_name]
    for planet1_deg in range(first_start, first_end):
        for planet2_deg in range(second_start, second_end):
            val = calc_combination_strength(planet1_name, planet2_name, planet1_deg, planet2_deg, start_val)
            for _ in range(20):
                randint = np.random.randint(0, 361, 9)
                row = [randint[0], randint[1], randint[2], randint[3], randint[4], randint[5], randint[6], randint[7], randint[8],
                        0 if val[1] < 150 else 1, text+val[0]]
                row[p1] = planet1_deg
                row[p2] = planet2_deg
                
                data_v2.append(row)


In [367]:

# Generate the data with for ascendant and moon combinations
def moonInHouses():
    skip_list = [8, 10] # skip when moon in Scorpio or Capricon
    for zn in range(1, 13):
        
        # Combinations of placement of Moon in houses 1,4,7,10
        combination2('Asc', 'Moon', zn, zn + 0, 340, skip_list, 'Asc conjunct Moon')
        combination2('Asc', 'Moon', zn, zn + 3, 330, skip_list, 'Moon in 4th House from Asc')
        combination2('Asc', 'Moon', zn, zn + 6, 330, skip_list, 'Moon in 7th House from Asc')
        combination2('Asc', 'Moon', zn, zn + 9, 330, skip_list, 'Moon in 10th House from Asc')

        # Combinations of placement of Moon in houses 5,9
        combination2('Asc', 'Moon', zn, zn + 4, 330, skip_list, 'Moon in 5th House from Asc')
        combination2('Asc', 'Moon', zn, zn + 8, 330, skip_list, 'Moon in 9th House from Asc')

         # Combinations of placement of Moon in houses 2,3,11
        combination2('Asc', 'Moon', zn, zn + 1, 300, skip_list, 'Moon in 2nd House from Asc')
        combination2('Asc', 'Moon', zn, zn + 2, 300, skip_list, 'Moon in 3rd House from Asc')
        combination2('Asc', 'Moon', zn, zn + 10, 315, skip_list, 'Moon in 11th House from Asc')
    
JupiterBeneficHouses = [[1, 7.5], [2, 7], [3, 7], [4, 10], [5, 8], [6, 7.5], [7, 7], [8, 7], [9, 9], [10, 7.5], [11, 7.5], [12, 9]]
def moonWithJupiterAspects():
    for kvp in JupiterBeneficHouses:
        zn = kvp[0]
        val = kvp[1]

        skip_list =  [] if(val > 8 ) else [8, 10] # skip when moon in Scorpio or Capricon
        # Jupiter aspect on Moon of 1,5,7, 9
        combination2('Jupiter', 'Moon', zn, zn + 0, 350, skip_list, 'Jupiter and Moon conjunct')
        combination2('Jupiter', 'Moon', zn, zn + 6, 345, skip_list, 'Jupiter with 7th aspect on Moon')
        combination2('Jupiter', 'Moon', zn, zn + 4, 340, skip_list, 'Jupiter with 5th aspect on Moon')
        combination2('Jupiter', 'Moon', zn, zn + 8, 340, skip_list, 'Jupiter with 9th aspect on Moon')

VenusBeneficHouses = [[1, 7.5], [2, 9], [3, 8], [4, 7], [5, 7], [6, 8], [7, 9], [8, 7.5], [9, 7.5], [10, 8], [11, 8], [12, 10]]
def moonWithVenusAspects():
    for kvp in VenusBeneficHouses:
        zn = kvp[0]
        val = kvp[1]

        skip_list =  [] if(val > 8 ) else [8, 10] # skip when moon in Scorpio or Capricon
        # Jupiter aspect on Moon of 1,5,7, 9
        combination2('Venus', 'Moon', zn, zn + 0, 350, skip_list, 'Venus and Moon conjunct')
        combination2('Venus', 'Moon', zn, zn + 6, 345, skip_list, 'Venus with 7th aspect on Moon')
        

SaturnMaleficHouses = [[1, 0.01], [2, 2.5], [3, 2.5], [4, 2.5], [5, 1], [6, 2.5], [7, 3.5], [8, 1], [9, 2.5], [10, 3.5], [11, 3.5], [12, 2.5]]
def moonWithSaturnAspects():
    for kvp in SaturnMaleficHouses:
        zn = kvp[0]
        val = kvp[1]

        skip_list =  [] 
        # Jupiter aspect on Moon of 1,5,7, 9
        combination2('Saturn', 'Moon', zn, zn + 0, 10, skip_list, 'Saturn and Moon conjunct')
        combination2('Saturn', 'Moon', zn, zn + 6, 25, skip_list, 'Saturn with 7th aspect on Moon')
        combination2('Saturn', 'Moon', zn, zn + 2, 60, skip_list, 'Saturn with 3rd aspect on Moon')
        combination2('Saturn', 'Moon', zn, zn + 9, 60, skip_list, 'Saturn with 10th aspect on Moon')

MarsMaleficHouses = [[1, 3.5], [2, 2.5], [3, 2.5], [4, 2.5], [5, 3], [6, 2.5], [7, 2.5], [8, 3.5], [9, 3.5], [10, 2.5], [11, 2.5], [12, 3.5]]
def moonWithMarsAspects():
    for kvp in MarsMaleficHouses:
        zn = kvp[0]
        val = kvp[1]

        skip_list =  [] 
        # Jupiter aspect on Moon of 1,5,7, 9
        combination2('Mars', 'Moon', zn, zn + 0, 10, skip_list, 'Mars and Moon conjunct')
        combination2('Mars', 'Moon', zn, zn + 6, 25, skip_list, 'Mars with 7th aspect on Moon')
        combination2('Mars', 'Moon', zn, zn + 3, 60, skip_list, 'Mars with 4th aspect on Moon')
        combination2('Mars', 'Moon', zn, zn + 7, 60, skip_list, 'Mars with 8th aspect on Moon')

RahuMaleficHouses = [[1, 1], [2, 3], [3, 3], [4, 0.01], [5, 1], [6, 3], [7, 2], [8, 1], [9, 2], [10, 2], [11, 2], [12, 1]]
def moonWithRahuAspects():
    for kvp in RahuMaleficHouses:
        zn = kvp[0]
        val = kvp[1]

        skip_list =  [] 
        # Jupiter aspect on Moon of 1,5,7, 9
        combination2('Jupiter', 'Moon', zn, zn + 0, 30, skip_list, 'Rahu and Moon conjunct')
        combination2('Jupiter', 'Moon', zn, zn + 6, 30, skip_list, 'Rahu with 7th aspect on Moon')
        combination2('Jupiter', 'Moon', zn, zn + 4, 30, skip_list, 'Rahu with 5th aspect on Moon')
        combination2('Jupiter', 'Moon', zn, zn + 8, 30, skip_list, 'Rahu with 9th aspect on Moon')

KetuMaleficHouses = [[1, 2], [2, 3], [3, 3], [4, 2], [5, 2], [6, 3], [7, 2], [8, 0.01], [9, 1], [10, 2], [11, 2], [12, 1]]
def moonWithKetuAspects():
    for kvp in KetuMaleficHouses:
        zn = kvp[0]
        val = kvp[1]

        skip_list =  [] 
        # Jupiter aspect on Moon of 1,5,7, 9
        combination2('Rahu', 'Moon', zn, zn + 10, 30, skip_list, 'Ketu with 5th aspect on Moon or Rahu with 11th')
        combination2('Rahu', 'Moon', zn, zn + 2, 30, skip_list, 'Ketu with 9th aspect on Moon or Rahu with 3rd')
      


In [368]:
def saveToExcel():
    columns = ['Asc','Sun','Moon','Mercury','Venus','Mars','Jupiter','Saturn','Rahu',
            #    'Sun_Retro','Jupiter_Retro','Mars_retro','Venus_Retro','Mercury_Retro',
            # 'is_Asc','is_Sun','is_Moon','is_Mercury','is_Venus','is_Mars','is_Jupiter','is_Saturn','is_Rahu',
               'calc_strength', 'Explanation']
    # Create the DataFrame with the new data
    df_v2 = pd.DataFrame(data_v2, columns=columns)

    # Identify duplicates in the first 18 columns
    df_deduplicated = df_v2.drop_duplicates(subset=df_v2.columns[:18], keep=False)
    print('duplicates', df_deduplicated.shape)
    df_v2 = df_deduplicated

    # Save to a new Excel file
    output_file_v2 = "data_v2.csv"
    df_v2.to_csv(output_file_v2, index=False)


moonInHouses()
moonWithJupiterAspects()
moonWithVenusAspects()
moonWithSaturnAspects()
moonWithMarsAspects()
moonWithRahuAspects()
moonWithKetuAspects()
saveToExcel()

0 31 0 31 340 Asc conjunct Moon. Moon in Aries and Asc in Aries
0 31 90 121 330 Moon in 4th House from Asc. Moon in Cancer and Asc in Aries
0 31 180 211 330 Moon in 7th House from Asc. Moon in Libra and Asc in Aries
Skipping as Moon in Capricorn
0 31 120 151 330 Moon in 5th House from Asc. Moon in Leo and Asc in Aries
0 31 240 271 330 Moon in 9th House from Asc. Moon in Sagittarius and Asc in Aries
0 31 30 61 300 Moon in 2nd House from Asc. Moon in Taurus and Asc in Aries
0 31 60 91 300 Moon in 3rd House from Asc. Moon in Gemini and Asc in Aries
0 31 300 331 315 Moon in 11th House from Asc. Moon in Aquarius and Asc in Aries
30 61 30 61 340 Asc conjunct Moon. Moon in Taurus and Asc in Taurus
30 61 120 151 330 Moon in 4th House from Asc. Moon in Leo and Asc in Taurus
Skipping as Moon in Scorpio
30 61 300 331 330 Moon in 10th House from Asc. Moon in Aquarius and Asc in Taurus
30 61 150 181 330 Moon in 5th House from Asc. Moon in Virgo and Asc in Taurus
Skipping as Moon in Capricorn
30 61 